In [1]:
from pathlib import Path
import pandas as pd
from itertools import product
from chan import detect_bi, analyze_bi_characteristics

ROOT = Path("/Users/cc/PycharmProjects/ChanLab")
DATA_CHAN_K = ROOT / "chan_data"
DATA_FRACTALS = DATA_CHAN_K / "fractals"

SYMBOLS = ["BTCUSDT", "ETHUSDT"]
TFS = ["5m", "15m", "30m", "1h", "4h"]

MIN_BARS = [3, 4, 5, 6]
MIN_DP = [0.005, 0.008, 0.01, 0.015]
USE_ATR_FILTER = [True, False]
ENABLE_BREAKTHROUGH = [True, False]

def run_bi_param_test(symbol, tf):
    df_k_path = DATA_CHAN_K / symbol / f"{tf}_chanK.csv"
    df_f_path = DATA_FRACTALS / symbol / f"{tf}_fractals.csv"

    if not df_k_path.exists() or not df_f_path.exists():
        print(f"[skip] missing data: {symbol} {tf}")
        return None

    df_k = pd.read_csv(df_k_path, parse_dates=["timestamp"])
    df_f = pd.read_csv(df_f_path, parse_dates=["timestamp"])

    records = []
    for mb, mdp, atrf, brk in product(MIN_BARS, MIN_DP, USE_ATR_FILTER, ENABLE_BREAKTHROUGH):
        try:
            bi = detect_bi(
                df_f, df_k,
                min_bars=mb,
                min_dp=mdp,
                use_relative_dp=True,
                use_atr_filter=atrf,
                enable_breakthrough=brk,
                enable_validation=True,
                enable_quality_markers=True
            )
            stat = analyze_bi_characteristics(bi)
            if not stat:
                continue
            rec = dict(
                symbol=symbol, tf=tf,
                min_bars=mb, min_dp=mdp,
                use_atr_filter=atrf,
                enable_breakthrough=brk,
                total=stat.get("total_bi", 0),
                up_down=stat.get("up_down_ratio", 0),
                avg_strength=stat.get("avg_strength", 0),
                avg_span=stat.get("avg_span", 0),
                max_strength=stat.get("max_strength", 0),
                min_strength=stat.get("min_strength", 0)
            )
            records.append(rec)
        except Exception as e:
            print(f"[error] {symbol} {tf} mb={mb}, mdp={mdp}, atr={atrf}, brk={brk}: {e}")

    return pd.DataFrame(records)


all_res = []
for s in SYMBOLS:
    for tf in TFS:
        df_res = run_bi_param_test(s, tf)
        if df_res is not None:
            all_res.append(df_res)

summary = pd.concat(all_res, ignore_index=True)
print(f"[done]  {len(summary)} ")

display(summary.head(20))

summary_group = (
    summary.groupby(["symbol", "tf"])
    .agg(
        total_mean=("total", "mean"),
        avg_strength=("avg_strength", "mean"),
        avg_span=("avg_span", "mean"),
        up_down=("up_down", "mean")
    )
    .reset_index()
)
display(summary_group)


[done] 共生成 640 条参数组合结果


symbol  tf  min_bars  min_dp  use_atr_filter  enable_breakthrough  total  \
0   BTCUSDT  5m         3   0.005            True                 True   2081   
1   BTCUSDT  5m         3   0.005            True                False   2300   
2   BTCUSDT  5m         3   0.005           False                 True   2084   
3   BTCUSDT  5m         3   0.005           False                False   2303   
4   BTCUSDT  5m         3   0.008            True                 True    808   
5   BTCUSDT  5m         3   0.008            True                False    878   
6   BTCUSDT  5m         3   0.008           False                 True    811   
7   BTCUSDT  5m         3   0.008           False                False    881   
8   BTCUSDT  5m         3   0.010            True                 True    458   
9   BTCUSDT  5m         3   0.010            True                False    493   
10  BTCUSDT  5m         3   0.010           False                 True    461   
11  BTCUSDT  5m         3   0.010           False                False    496   
12  BTCUSDT  5m         3   0.015            True                 True    145   
13  BTCUSDT  5m         3   0.015            True                False    160   
14  BTCUSDT  5m         3   0.015           False                 True    147   
15  BTCUSDT  5m         3   0.015           False                False    162   
16  BTCUSDT  5m         4   0.005            True                 True   1815   
17  BTCUSDT  5m         4   0.005            True                False   1983   
18  BTCUSDT  5m         4   0.005           False                 True   1817   
19  BTCUSDT  5m         4   0.005           False                False   1985   

     up_down  avg_strength  avg_span  max_strength  min_strength  
0   1.028265      0.008901  7.249880      0.129435      0.005004  
1   1.012248      0.008696  7.073043      0.129435      0.005003  
2   1.027237      0.008910  7.244722      0.129435      0.005004  
3   1.011354      0.008705  7.068606      0.129435      0.005003  
4   1.071795      0.012829  8.103960      0.129435      0.008004  
5   1.100478      0.012624  7.962415      0.129435      0.008004  
6   1.068878      0.012839  8.087546      0.129435      0.008004  
7   1.097619      0.012634  7.947787      0.129435      0.008004  
8   1.081818      0.015732  8.550218      0.129435      0.010015  
9   1.088983      0.015464  8.446247      0.129435      0.010015  
10  1.076577      0.015730  8.518438      0.129435      0.010015  
11  1.084034      0.015464  8.417339      0.129435      0.010015  
12  1.164179      0.022631  9.241379      0.129435      0.015038  
13  1.222222      0.022226  9.287500      0.129435      0.015038  
14  1.161765      0.022559  9.163265      0.129435      0.015038  
15  1.219178      0.022166  9.216049      0.129435      0.015038  
16  1.053167      0.009055  7.767493      0.129435      0.005004  
17  1.027607      0.008837  7.646495      0.129435      0.005004  
18  1.053107      0.009062  7.763346      0.129435      0.005004  
19  1.027579      0.008843  7.642821      0.129435      0.005004

symbol   tf   total_mean  avg_strength  avg_span   up_down
0  BTCUSDT  15m   604.375000      0.016679  8.267797  1.077079
1  BTCUSDT   1h   293.843750      0.023912  7.167077  1.159235
2  BTCUSDT  30m   461.656250      0.019525  7.526064  1.079687
3  BTCUSDT   4h    84.453125      0.040349  6.788136  1.032259
4  BTCUSDT   5m   752.125000      0.014991  8.942967  1.101283
5  ETHUSDT  15m  1126.406250      0.022196  7.173638  1.085959
6  ETHUSDT   1h   340.968750      0.039650  6.882674  1.187288
7  ETHUSDT  30m   659.656250      0.028633  6.972813  1.126206
8  ETHUSDT   4h    94.468750      0.079862  6.911144  1.118264
9  ETHUSDT   5m  2038.906250      0.017236  7.626590  1.071705

In [2]:
from pathlib import Path
import pandas as pd, json
from chan import detect_fractals, detect_bi

ROOT = Path("/Users/cc/PycharmProjects/ChanLab")
DATA_CHAN_K = ROOT / "chan_data"
DATA_OUT = ROOT / "chan_data/bi"
CONFIGS = ROOT / "configs"

# === 参数加载 ===
with open(CONFIGS / "fractals_default.json", "r") as f:
    FRACTAL_PARAMS = json.load(f)
    FRACTAL_PARAMS.pop("mode", None)
with open(CONFIGS / "bi_default.json", "r") as f:
    BI_PARAMS = json.load(f)

SYMBOLS = ["BTCUSDT", "ETHUSDT"]
TFS = ["5m", "15m", "30m", "1h", "4h"]

DATA_OUT.mkdir(exist_ok=True)

def process_symbol_tf(symbol: str, tf: str):
    k_path = DATA_CHAN_K / symbol / f"{tf}_chanK.csv"
    if not k_path.exists():
        print(f"[skip] missing {k_path}")
        return None

    try:
        df_k = pd.read_csv(k_path, parse_dates=["timestamp"])

        fr = detect_fractals(df_k, **FRACTAL_PARAMS)
        if fr.empty:
            print(f"[warn] no fractals {symbol}-{tf}")
            return None

        bi = detect_bi(fr, df_k, **BI_PARAMS)
        if bi.empty:
            print(f"[warn] no bi {symbol}-{tf}")
            return None

        out_dir = DATA_OUT / symbol
        out_dir.mkdir(parents=True, exist_ok=True)
        out_path = out_dir / f"{tf}_bi.csv"
        bi.to_csv(out_path, index=False)

        print(f"[saved] {symbol}-{tf}: {len(bi)} 笔 -> {out_path}")
        return bi

    except Exception as e:
        print(f"[error] {symbol}-{tf}: {e}")
        return None


for s in SYMBOLS:
    for tf in TFS:
        process_symbol_tf(s, tf)

import numpy as np

records = []
for s in SYMBOLS:
    for tf in TFS:
        p = DATA_OUT / s / f"{tf}_bi.csv"
        if not p.exists():
            continue
        df = pd.read_csv(p)
        if df.empty:
            continue
        rec = dict(
            symbol=s, tf=tf,
            total=len(df),
            avg_span=np.round(df["k_bars_span"].mean(), 2),
            avg_strength=np.round(df["strength"].mean(), 4),
            up_down_ratio=np.round(
                (df["dir"] == 1).sum() / max(1, (df["dir"] == -1).sum()), 3
            )
        )
        records.append(rec)

summary = pd.DataFrame(records)
display(summary)


[saved] BTCUSDT-5m: 433 笔 -> /Users/cc/PycharmProjects/ChanLab/chan_data/bi/BTCUSDT/5m_bi.csv
[saved] BTCUSDT-15m: 484 笔 -> /Users/cc/PycharmProjects/ChanLab/chan_data/bi/BTCUSDT/15m_bi.csv
[saved] BTCUSDT-30m: 423 笔 -> /Users/cc/PycharmProjects/ChanLab/chan_data/bi/BTCUSDT/30m_bi.csv
[saved] BTCUSDT-1h: 293 笔 -> /Users/cc/PycharmProjects/ChanLab/chan_data/bi/BTCUSDT/1h_bi.csv
[saved] BTCUSDT-4h: 83 笔 -> /Users/cc/PycharmProjects/ChanLab/chan_data/bi/BTCUSDT/4h_bi.csv
[saved] ETHUSDT-5m: 1686 笔 -> /Users/cc/PycharmProjects/ChanLab/chan_data/bi/ETHUSDT/5m_bi.csv
[saved] ETHUSDT-15m: 1125 笔 -> /Users/cc/PycharmProjects/ChanLab/chan_data/bi/ETHUSDT/15m_bi.csv
[saved] ETHUSDT-30m: 668 笔 -> /Users/cc/PycharmProjects/ChanLab/chan_data/bi/ETHUSDT/30m_bi.csv
[saved] ETHUSDT-1h: 337 笔 -> /Users/cc/PycharmProjects/ChanLab/chan_data/bi/ETHUSDT/1h_bi.csv
[saved] ETHUSDT-4h: 97 笔 -> /Users/cc/PycharmProjects/ChanLab/chan_data/bi/ETHUSDT/4h_bi.csv


symbol   tf  total  avg_span  avg_strength  up_down_ratio
0  BTCUSDT   5m    433      8.86        0.0157          1.112
1  BTCUSDT  15m    484      7.96        0.0172          1.077
2  BTCUSDT  30m    423      7.30        0.0196          1.034
3  BTCUSDT   1h    293      6.84        0.0238          1.187
4  BTCUSDT   4h     83      6.57        0.0404          0.976
5  ETHUSDT   5m   1686      7.42        0.0176          1.064
6  ETHUSDT  15m   1125      6.80        0.0218          1.072
7  ETHUSDT  30m    668      6.64        0.0287          1.134
8  ETHUSDT   1h    337      6.40        0.0394          1.203
9  ETHUSDT   4h     97      6.31        0.0805          0.980